In [8]:
pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pip in c:\users\aaaimeeeelll\appdata\roaming\python\python312\site-packages (24.3.1)



In [2]:
pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached matplotlib-3.9.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.7-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-11.0.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached matplotlib-3.9.2-cp312-cp312-win_amd64.whl (7.8 MB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 17.8 MB/s eta 0:00:

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Python312\\share'
Consider using the `--user` option or check the permissions.



In [3]:
pip install scikit-learn-extra

     ---------------------------------------- 0.0/819.0 kB ? eta -:--:--
     -------------------------------------- 819.0/819.0 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build scikit-learn-extra
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for scikit-learn-extra (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [60 lines of output]
      E:\Temp\pip-build-env-hm3epjds\overlay\Lib\site-packages\setuptools\dist.py:488: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      
              By 2025-Mar-03, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              ********************************************************************************
      
      !!
        opt = self.warn_

### Helper Functions: Algorithms & Clustering

**Algorithms**

DONE: add a "mode" argument to each algorithm that, if mode = 1 the cluster labelling is output and if mode = 0 the silhouette coefficient is output.

In [4]:
# Code for KMeans

import numpy as np
from sklearn.cluster import KMeans
from scipy.stats import multivariate_normal
from sklearn.metrics import silhouette_score

def kmeans_clustering(samples,mode,  n_clusters=2, max_iter=300):
    """
    Perform KMeans clustering on the input samples
    
    Parameters:
        samples: array-like, shape (n_samples, n_features)
        n_clusters: int, number of clusters (default=2)
        max_iter: int, maximum iterations (default=300)
    
    Returns:
        silhouette_coef: silhouette coefficient score
    """
    k_means = KMeans(n_clusters=n_clusters, max_iter=max_iter)
    k_means.fit(samples)
    if mode == 0:
        try:
            silhouette_coef = silhouette_score(samples, k_means.labels_, metric='euclidean')
        except ValueError:
            silhouette_coef = 0  # Assigning lowest score if clustering fails
        return silhouette_coef
    if mode == 1:
        return k_means.labels_

In [5]:
# EM Clustering Code

from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

def em_clustering(selected_features, mode, n_clusters=2):
    """
    Perform EM Clustering on selected features and return silhouette score.
        
    Returns:
    --------
    float
        Silhouette score of the clustering (-1 if clustering fails)
    """
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Initialize and fit the EM model
    em_model = GaussianMixture(
        n_components=n_clusters,
        random_state=0, #THOUGHTS: We can improve this later to have an array of seeds to select from to observe variations
        n_init=10  # Multiple initializations to avoid local optima
    )
    
   
    try:
        # Fit the model and get cluster assignments
        em_model.fit(X_scaled)
        labels = em_model.predict(X_scaled)
        
        # Calculate silhouette score
        silhouette_coef = silhouette_score(X_scaled, labels)
    except Exception as e:
        #print(f"Clustering failed: {str(e)}")
        silhouette_coef = 0  # Assigning lowest score if clustering fails
    if mode == 0:
        return silhouette_coef
    if mode == 1:
        return labels

In [6]:
# DBSCAN Detection method: 
# I put 'optimization part' in 'DBSCAN_Optimization_Code.ipynb' file. 
# We can use optimization after initial run to do a comparison and analysis in our paper to show improvements.

import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# TO-DO: keep -1 --> they will be its own cluster
def dbscan_clustering(selected_features, mode, eps=0.5, min_samples=5):
    """
    Perform DBSCAN clustering on selected features
    
    Parameters:
    selected_features : pandas DataFrame
        The features selected for clustering
    eps : float
        The maximum distance between two samples for them to be considered neighbors
    min_samples : int
        The number of samples in a neighborhood for a point to be considered a core point
        
    Returns:
    float : silhouette coefficient
    dict : additional clustering information
    """

    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Initialize and fit DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X_scaled)
    
    # Get number of clusters (excluding noise points which are labeled -1, K Medoids does not have noise points)
    n_clusters = len(set(labels))

    if -1 in labels:
        labels[labels == -1] = n_clusters - 1
    
    # calculate silhouette score if more than one cluster and  noise points
    if n_clusters > 1:
        silhouette_coef = silhouette_score(X_scaled, labels)
    else:
        silhouette_coef = 0  # Assign lowest score if clustering fails

    
    # NOTE: -- Uncomment when we analyze and optimize ---- Additional clustering information
    # info = {
    #     'n_clusters': n_clusters,
    #     'n_noise': list(labels).count(-1),
    #     'labels': labels,
    #     'cluster_sizes': pd.Series(labels).value_counts().to_dict()
    # }
    
    if mode == 0:
        return silhouette_coef
    if mode == 1:
        return labels


In [7]:
# Code for K Medoids
import numpy as np
import pandas as pd

from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

def kmedoids_clustering(selected_features, mode, n_clusters=2):
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

     # Initialize and fit the K-Medoids model
    kmedoids = KMedoids(n_clusters=n_clusters, method='pam', max_iter=1000, random_state=0)
    labels = kmedoids.fit_predict(X_scaled)

    # Calculate silhouette score
    try:
        silhouette_coef = silhouette_score(X_scaled, labels)
    except ValueError:
        silhouette_coef = 0  # Assigning lowest score if clustering fails
    if mode == 0:
        return silhouette_coef
    if mode == 1:
        return labels

In [8]:
# Codes for Mean Shift
import numpy as np
import pandas as pd

from sklearn.cluster import MeanShift
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

def meanshift_clustering(selected_features, mode, bandwidth=None):
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Initialize and fit the Mean Shift model
    meanshift = MeanShift(bandwidth=bandwidth)
    labels = meanshift.fit_predict(X_scaled)

    # Check the number of clusters determined 
    n_clusters = len(np.unique(labels))
    #print(f"Number of clusters found: {n_clusters}")
    
    
    # Calculate silhouette score
    try:
        silhouette_coef = silhouette_score(X_scaled, labels)
    except ValueError:
        silhouette_coef = 0  # Assign lowest score if clustering fails
    if mode == 0:
        return silhouette_coef
    if mode == 1:
        return labels

**Clustering & Output**

### Main Code

In [ ]:
# # idx = (np.where(percentages <= 0.1)[0]).tolist()
# idx = []
# anomalies = labelled_data.loc[labelled_data['cluster'].isin(idx)]
# print(len(anomalies))
perc_values = labelled_data['cluster'].value_counts(ascending=True)
print(perc_values)
print("flipped:",perc_values.values)
print(percentages)
idx = (np.where(percentages <= 0.1)[0]).tolist()
print(idx)
anomalies = labelled_data.loc[labelled_data['cluster'].isin(idx)]
print(len(anomalies))
perc_values = labelled_data['cluster'].value_counts().values

In [3]:
import pandas as pd
import numpy as np

FEATURES = {0: 'avg_bytes_sent', 1: 'avg_bytes_received', 2: 'avg_packets_transferred', 
  3: 'avg_flow_duration', 4: 'recent_tcp_flags', 5: 'recent_protocol', 6: 'avg_cpu_usage', 
  7: 'avg_memory_usage', 8: 'avg_disk_usage', 9: 'avg_uptime'}

data = pd.read_csv("joined_quantitative_data.csv")

ALGORITHMS = {4: 'K-Means', 1: 'Mean Shift', 2: 'K-Mediods', 3: 'EM Clustering', 0: 'DBSCAN Clustering'}
NUM_ALG = len(ALGORITHMS)
original_features = data.iloc[:, 2:]
ips = data['source_ip']
#print(original_features.columns)
#print(ips.head(10))

def algorithm_prep(state, action, mode):
  # convert state to binary
  state_bin = bin(state)
  #print(state_bin)
  state_bin_arr = np.array([b for b in state_bin[2:]])
  # pad with zeros
  diff = 10 - len(state_bin_arr)
  padded_arr = np.insert(state_bin_arr, 0, ['0' for i in range(diff)])
  #(padded_arr)
  # identify which indexes are 1
  idx = (np.where(padded_arr == '1')[0]).tolist()
  #print(idx)
  # select feature headings
  selected_features = original_features.iloc[:,idx]
  #print(selected_features.head(10))
  # select algorithm
  # algo = action
  # prep correct data - done
  
  # call algorithm function
  out = None
  #print('algorithm:',ALGORITHMS[action])

  # if mode = 0, output is the silhouette coefficient
  # if mode = 1, output is the cluster labelling
  match action:   
    case 0:
      #print('algorithm:',ALGORITHMS[action])
      out = dbscan_clustering(selected_features, mode)
    case 1: 
      #print('algorithm:',ALGORITHMS[action])
      out = meanshift_clustering(selected_features, mode)
    case 2:
      #print('algorithm:',ALGORITHMS[action])
      out = kmedoids_clustering(selected_features, mode)
    case 3: 
      out = em_clustering(selected_features, mode)
    case 4:
      out = kmeans_clustering(selected_features, mode)
      
  # return silhouette from algorithm function
  return out

In [11]:
def bin_to_features(state):
  state_bin = bin(state)
  #print(state_bin)
  state_bin_arr = np.array([b for b in state_bin[2:]])
  # pad with zeros
  diff = 10 - len(state_bin_arr)
  padded_arr = np.insert(state_bin_arr, 0, ['0' for i in range(diff)])
  #(padded_arr)
  # identify which indexes are 1
  idx = (np.where(padded_arr == '1')[0]).tolist()
  #print(idx)
  # select feature headings
  selected_features = original_features.iloc[:,idx]
  features = selected_features.columns.tolist()
  print("Features Used:", features)
  return


In [30]:
bin_to_features(48)

Features Used: ['recent_tcp_flags', 'recent_protocol']


##### Sampling

In [19]:
label_test = algorithm_prep(256, 4,1)
labelled_data = data.copy()
labelled_data['cluster'] = label_test
labelled_data.head(10)

num_clusters = labelled_data['cluster'].nunique()
vals = labelled_data['cluster'].value_counts().values
print(labelled_data.shape[0])
vals = vals / labelled_data.shape[0]
idx = (np.where(vals <= 0.1)[0]).tolist()
print(idx)
anomalies = labelled_data.loc[labelled_data['cluster'].isin(idx)]
#print(anomalies)
for i in anomalies['source_ip']:
    print(f"IP {i} is a potential anomaly")

297
[1]
IP 192.168.0.248 is a potential anomaly
IP 192.168.0.78 is a potential anomaly


In [7]:
labelled_data[labelled_data['cluster']==1]

NameError: name 'labelled_data' is not defined

#### Reinforcement Learning

In [ ]:
# Markov Decision Process (MDP) - The Bellman equations adapted to
# Q Learning.Reinforcement Learning with the Q action-value(reward) function.
# Copyright 2018 Denis Rothman MIT License. See LICENSE.
import numpy as ql
# R is The Reward Matrix for each state
# 1024 configurations of the 10 features --> 2^10
# 5 algorithms
R = ql.matrix(ql.zeros([1024,5]))

# Q is the Learning Matrix in which rewards will be learned/stored
Q = ql.matrix(ql.zeros([1024,5]))

# Gamma : It's a form of penalty or uncertainty for learning
# If the value is 1 , the rewards would be too high.
# This way the system knows it is learning.
gamma = 0.8

# agent_s_state. The agent the name of the system calculating
# s is the state the agent is going from and s' the state it's going to
# this state can be random or it can be chosen as long as the rest of the choices
# are not determined. Randomness is part of this stochastic process
# 1) DONE: decide if starting state is random or a specific state
agent_s_state = 1

# The possible "a" actions when the agent is in a given state
def possible_actions(state):
    # 2) DONE: we should check Q, not R because R is never modified
    current_state_row = Q[state,]
    # 3) DONE: this should pick valid actions based on what we have not visited
    possible_act = ql.where(current_state_row == 0)[1]
    return possible_act

# Get available actions in the current state
PossibleAction = possible_actions(agent_s_state)

# This function chooses at random which action to be performed within the range 
# of all the available actions.
def ActionChoice(available_actions_range):
    if(sum(PossibleAction)>0):
        next_action = int(ql.random.choice(PossibleAction,1)[0])
    if(sum(PossibleAction)<=0):
        next_action = int(np.random.choice(NUM_ALG,1)[0])
    return next_action

# Sample next action to be performed
action = ActionChoice(PossibleAction)

# A version of Bellman's equation for reinforcement learning using the Q function
# This reinforcement algorithm is a memoryless process
# The transition function T from one state to another
# is not in the equation below.  T is done by the random choice above

def reward(current_state, action, gamma):
    Max_State = ql.where(Q[action,] == ql.max(Q[action,]))[1]

    if Max_State.shape[0] > 1:
        Max_State = int(ql.random.choice(Max_State, size = 1)[0])
    else:
        Max_State = int(Max_State[0])

    # 5) DONE: we think this is a typo and action/Max_State should be switched. 
    # MaxValue = Q[action, Max_State]
    MaxValue = Q[Max_State, action]

    # 6) DONE: call function to run ML algorithm using the value of action. this will
    # run the algorithm using the features from current_state, create clusters,
    # and calculate the silhouette value.
    silhouette_co = algorithm_prep(current_state, action, 0) 
    
    # Bellman's MDP based Q function
    # 7) DONE: instead of getting a value from R, we add the silhouette value to gamma * MaxValue
    # Q[current_state, action] = R[current_state, action] + gamma * MaxValue
    Q[current_state, action] = silhouette_co + gamma * MaxValue


# Rewarding Q matrix
reward(agent_s_state,action,gamma)


# Leraning over n iterations depending on the convergence of the system
# A convergence function can replace the systematic repeating of the process
# by comparing the sum of the Q matrix to that of Q matrix n-1 in the
# previous episode
for i in range(6000):
    # select a random new state (configuration of features)
    current_state = ql.random.randint(1, int(Q.shape[0]))
    PossibleAction = possible_actions(current_state)
    action = ActionChoice(PossibleAction)
    reward(current_state,action,gamma)
    
# Displaying Q before the norm of Q phase
print("Q  :")
print(Q)

# Norm of Q
print("Normed Q :")
print(Q/ql.max(Q)*100)

# DONE: get maximum value from Q-Learning Matrix
normed_Q = Q/ql.max(Q)*100
max_location = np.where(normed_Q==normed_Q.max())
print("\nmax value located at",max_location)
max_config = max_location[0][0]
max_algorithm = max_location[1][0]
final_feats = bin_to_features(max_config)
print(f"\nUsing algorithm {ALGORITHMS[max_algorithm]} and {final_feats}, max value is:",normed_Q[max_config,max_algorithm])
#TO-DO: print(f"Selected features:", )

# DONE: get final cluster labels
cluster_labels = algorithm_prep(max_config, max_algorithm, 1)

# DONE: match data in clusters to IP addresses
labelled_data = data.copy()
labelled_data['cluster'] = cluster_labels

# TO-DO: return what IPs are likely anomalous
# see what clusters have < 5% of the data
# get unique values in cluster column
num_clusters = labelled_data['cluster'].nunique()

# for each unique value, get the count / len of data (aka percentage)
# num_clusters = labelled_data['cluster'].nunique()
cluster_array = labelled_data['cluster'].to_numpy()
perc_values = np.unique(cluster_array,return_counts = True)[-1]
percentages = perc_values / labelled_data.shape[0]

# keep cluster values with % < 5
idx = (np.where(percentages <= 0.1)[0]).tolist()
anomalies = labelled_data.loc[labelled_data['cluster'].isin(idx)]
# output IPs within those selected clusters
#print(f"There are {len(an)}")
for i in anomalies['source_ip']:
    print(f"\nIP {i} is a potential anomaly")

/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceW

Q  :
[[0.         0.         0.         0.         0.        ]
 [0.         0.         0.63071706 0.63071706 0.63040802]
 [0.         0.         0.         0.         0.        ]
 ...
 [0.         0.         0.13145184 0.         1.81181496]
 [0.         0.69277059 0.1199422  0.7984389  1.81181623]
 [0.         0.         0.11100989 0.78066054 1.81181495]]
Normed Q :
[[ 0.          0.          0.          0.          0.        ]
 [ 0.          0.         34.31096091 34.31096091 34.29414902]
 [ 0.          0.          0.          0.          0.        ]
 ...
 [ 0.          0.          7.15097043  0.         98.56259762]
 [ 0.         37.68666811  6.52484694 43.43501649 98.56266685]
 [ 0.          0.          6.03892973 42.46787466 98.56259722]]

max value located at (array([32, 48]), array([4, 4]))

Using algorithm K-Means and feature configuration 32, max value is: 100.0

IP 192.168.0.0 is a potential anomaly

IP 192.168.0.1 is a potential anomaly

IP 192.168.0.10 is a potential anomal

#### ignore

In [27]:
import numpy as np
# percentages = labelled_data['cluster'].value_counts().values
# percentages = percentages / labelled_data.shape[0]
# print(percentages)
# print(labelled_data['cluster'].value_counts())
cluster_array = labelled_data['cluster'].to_numpy()
perc_vals = np.unique(cluster_array,return_counts = True)[-1]
print(perc_vals)
percentages = perc_vals / labelled_data.shape[0]
print(percentages)
idx = (np.where(percentages <= 0.1)[0]).tolist()
anomalies = labelled_data.loc[labelled_data['cluster'].isin(idx)]
print(anomalies)

[296   1]
[0.996633 0.003367]
    device_name      source_ip  avg_bytes_sent  avg_bytes_received  \
142  Device-227  192.168.0.226          9973.5              4648.0   

     avg_packets_transferred  avg_flow_duration  recent_tcp_flags  \
142                   303.75            7694.75                 2   

     recent_protocol  avg_cpu_usage  avg_memory_usage  avg_disk_usage  \
142                1         29.545            58.905           58.96   

     avg_uptime  cluster  
142       426.0        1  


In [ ]:
#print(max_location)
np.where(normed_Q==normed_Q.max())
#normed_Q[normed_Q > 99.]
#max_location = np.where(normed_Q==normed_Q.max())

matrix([[ 99.99998773, 100.        ,  99.99998111,  99.9512457 ,
          99.9512457 ,  99.95124659,  99.99917226,  99.99991577,
          99.9991523 ,  99.99991297,  99.95122339,  99.95121864]])

In [101]:
normed_Q = Q/ql.max(Q)*100
max_location = np.where(normed_Q==normed_Q.max())
print("max value located at",max_location)
max_config = max_location[0][0]
max_algorithm = ALGORITHMS[max_location[1][0]]
print(f"Using algorithm {max_algorithm} and feature configuration {max_config}, max value is:",normed_Q[278,0])



max value located at (array([278]), array([0]))
278
Using algorithm K-Means and feature configuration 278, max value is: 100.0


## Additional Reference Code

In [4]:
# -*- coding: utf-8 -*-
# Markov Decision Process (MDP) - The Bellman equations adapted to
# Q Learning.Reinforcement Learning with the Q action-value(reward) function.
# Copyright 2019 Denis Rothman MIT License. See LICENSE.
import numpy as ql
# R is The Reward Matrix for each state
R = ql.matrix([ [0,0,0,0,1,0],
		            [0,0,0,1,0,1],
		            [0,0,100,1,0,0],
	             	[0,1,1,0,1,0],
		            [1,0,0,1,0,0],
		            [0,1,0,0,0,0] ])

# Q is the Learning Matrix in which rewards will be learned/stored
Q = ql.matrix(ql.zeros([6,6]))

"""##  The Learning rate or training penalty"""

# Gamma : It's a form of penalty or uncertainty for learning
# If the value is 1 , the rewards would be too high.
# This way the system knows it is learning.
gamma = 0.8

"""## Initial State"""

# agent_s_state. The agent the name of the system calculating
# s is the state the agent is going from and s' the state it's going to
# this state can be random or it can be chosen as long as the rest of the choices
# are not determined. Randomness is part of this stochastic process
agent_s_state = 5

"""## The random choice of the next state"""

# The possible "a" actions when the agent is in a given state
def possible_actions(state):
    current_state_row = R[state,]
    possible_act = ql.where(current_state_row >0)[1]
    return possible_act

# Get available actions in the current state
PossibleAction = possible_actions(agent_s_state)

# This function chooses at random which action to be performed within the range 
# of all the available actions.
def ActionChoice(available_actions_range):
    if(sum(PossibleAction)>0):
        next_action = int(ql.random.choice(PossibleAction,1))
    if(sum(PossibleAction)<=0):
        next_action = int(ql.random.choice(5,1))
    return next_action

# Sample next action to be performed
action = ActionChoice(PossibleAction)

"""## The Bellman Equation"""

# A version of the Bellman equation for reinforcement learning using the Q function
# This reinforcement algorithm is a memoryless process
# The transition function T from one state to another
# is not in the equation below.  T is done by the random choice above

def reward(current_state, action, gamma):
    Max_State = ql.where(Q[action,] == ql.max(Q[action,]))[1]

    if Max_State.shape[0] > 1:
        Max_State = int(ql.random.choice(Max_State, size = 1))
    else:
        Max_State = int(Max_State)
    MaxValue = Q[action, Max_State]
    
    # The Bellman MDP based Q function
    Q[current_state, action] = R[current_state, action] + gamma * MaxValue

# Rewarding Q matrix
reward(agent_s_state,action,gamma)

"""## Running the training episodes randomly"""

# Learning over n iterations depending on the convergence of the system
# A convergence function can replace the systematic repeating of the process
# by comparing the sum of the Q matrix to that of Q matrix n-1 in the
# previous episode
for i in range(50000):
    current_state = ql.random.randint(0, int(Q.shape[0]))
    PossibleAction = possible_actions(current_state)
    action = ActionChoice(PossibleAction)
    reward(current_state,action,gamma)
    
# Displaying Q before the norm of Q phase
print("Q  :")
print(Q)

# Norm of Q
print("Normed Q :")
print(Q/ql.max(Q)*100)

"""# Improving the program by introducing a decision-making process"""
nextc=-1
nextci=-1
conceptcode=["A","B","C","D","E","F"]
origin=int(input("index number origin(A=0,B=1,C=2,D=3,E=4,F=5): "))
print("Concept Path")
print("->",conceptcode[int(origin)])
for se in range(0,6):
    if(se==0):
        po=origin
    if(se>0):
        po=nextci
        #print("se:",se,"po:",po)
    for ci in range(0,6):
        maxc=Q[po,ci]
        #print(maxc,nextc)
        if(maxc>=nextc):
            nextc=maxc
            nextci=ci
            #print("next c",nextc)
    if(nextci==po):
        break;
    #print("present origin",po,"next c",nextci," ",nextc," ",conceptcode[int(nextci)])
    print("->",conceptcode[int(nextci)])


Q  :
[[  0.      0.      0.      0.    258.44    0.   ]
 [  0.      0.      0.    321.8     0.    207.752]
 [  0.      0.    500.    321.8     0.      0.   ]
 [  0.    258.44  401.      0.    258.44    0.   ]
 [207.752   0.      0.    321.8     0.      0.   ]
 [  0.    258.44    0.      0.      0.      0.   ]]
Normed Q :
[[  0.       0.       0.       0.      51.688    0.    ]
 [  0.       0.       0.      64.36     0.      41.5504]
 [  0.       0.     100.      64.36     0.       0.    ]
 [  0.      51.688   80.2      0.      51.688    0.    ]
 [ 41.5504   0.       0.      64.36     0.       0.    ]
 [  0.      51.688    0.       0.       0.       0.    ]]
Concept Path
-> A
-> E
-> D
-> C
